In [29]:
%load_ext sql

In [30]:
%config SqlMagic.displaylimit = 15
# %config SqlMagic.displaylimit = None

In [65]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

In [42]:
sys_log_db = "system_data_log.db"

sql_engine = create_engine("sqlite:///%s" % sys_log_db, echo=False)

In [ ]:
def create_voltage_table(force=False):
    with sql_engine.connect() as sql_conn:
        if force:
            sql_text = text("""DROP TABLE IF EXISTS topic;
                            """)
            sql_conn.execute(sql_text)
        # define schema
        sql_text = text("""
            CREATE TABLE IF NOT EXISTS voltages (
                timestamp TEXT,
                Vmain FLOAT,
                Vmain_raw FLOAT,
                Vaux FLOAT,
                Vaux_raw FLOAT,
                PRIMARY KEY (timestamp)
                )
        """)
        sql_conn.execute(sql_text)
        sql_conn.commit()

In [53]:
create_voltage_table()

In [ ]:
def add_voltage_log(Vmain, Vmain_raw, Vaux, Vaux_raw):
    with sql_engine.connect() as sql_conn:
        sql_text = text("""INSERT INTO voltages
                           VALUES (DateTime("now", "localtime"),
                                   :v_main,
                                   :v_main_raw,
                                   :v_aux,
                                   :v_aux_raw
                           );
                        """)
        sql_conn.execute(sql_text, {"v_main": Vmain,
                                    "v_main_raw": Vmain_raw,
                                    "v_aux": Vaux,
                                    "v_aux_raw": Vaux_raw})
        sql_conn.commit()

In [ ]:
add_voltage_log(12.4, 12.8, 12.5, 12)

In [68]:
add_voltage_log(12.5, 12.75, 12.4, 11.9)

In [ ]:
# read
def get_voltages_all():
    with sql_engine.connect() as sql_conn:
        sql_text = text("""SELECT *
                            FROM voltages
                        """)
        result_df = pd.read_sql(sql_text, con=sql_conn)
    return result_df

In [69]:
get_voltages_all()

,timestamp,V_main,V_main_raw,V_aux,V_aux_raw
0,2025-10-12 13:07:13,12.4,12.80,12.5,12.0
1,2025-10-12 13:09:58,12.5,12.75,12.4,11.9


---

In [134]:
# works
def get_voltages(voltage_type="V_main"):
    with sql_engine.connect() as sql_conn:
        # sql_text = text("""SELECT V_main
        sql_text = text("""SELECT {}
                            FROM voltages
                        """.format(voltage_type))
        result_df = pd.read_sql(sql_text, con=sql_conn)
        # result_df = pd.read_sql(sql_text, con=sql_conn,
                                # parse_dates={"date_column": {"format": "%d/%m/%y"}})
                                # params={"Vtype": voltage_type},
    return result_df

In [ ]:
def get_voltages(voltage_type="V_main"):
    with sql_engine.connect() as sql_conn:
        # sql_text = text("""SELECT V_main
        sql_text = text("""SELECT "V_main"
                            FROM voltages
                        """)
        result_df = pd.read_sql(sql_text, con=sql_conn, params={"Vtype": voltage_type})
    return result_df

In [137]:
get_voltages()

,V_main
0,12.4
1,12.5


---

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
pandas.DataFrame.to_sql(sys_log_db, )